In [1]:
import pandas as pd
import numpy as np
import boto3
import sys

# ==============================================================
# 승호 S3로부터 CSV 파일 불러오기.
if sys.version_info[0] < 3:
    from io import StringIO # Python 2.x
else:
    from io import StringIO # Python 3.x

# get your credentials from environment variables
aws_id = ''
aws_secret = ''

client = boto3.client('s3', aws_access_key_id=aws_id,
        aws_secret_access_key=aws_secret)

bucket_name = 'snucsv'

object_key = 'SNU_FastText_1200.csv'
csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
body = csv_obj['Body']
csv_string = body.read().decode('cp949')

object_key2 = 'SNU_Data_1200.csv'
csv_obj2 = client.get_object(Bucket=bucket_name, Key=object_key2)
body2 = csv_obj2['Body']
csv_string2 = body2.read().decode('cp949')

df = pd.read_csv(StringIO(csv_string))
df2 = pd.read_csv(StringIO(csv_string2))
df
df2

,document,label
0,현재 AI기술은 불완전하기 때문에 가짜뉴스 완전히 걸러낼 수 없다,1
1,"‘정치 세대교체’ 바람 일으킨 이준석, 대선엔 출마 못한다",1
2,액상형 전자담배는 니코틴이 함유되지 않아 담배가 아니다,1
3,한국 백신 접종률 세계 100위권 이하다,1
4,광역도시 없는 지역권은 실제 수요와 관계없이 획일적으로 국가교통망 정책에서 소외되고 있다,1
...,...,...
1219,""" ‘당·정·청 전원회의’는 '운동권 용어'다""?",0
1220,"""늘어가는 해양사고, 전북 관내 잠수구조인력 전무(全無)""?",0
1221,"""청와대 하루 평균 업무추진비, 이명박 768만원, 박근혜 814만원, 문재인 55만원""",0
1222,"국민연금공단 기금운용본부가 전주로 이전한 후, 해외투자자가 국민연금을 '패싱'한다",0


In [2]:
df_train = df.fillna(0)
df_mul = df_train.mul(1000)

In [3]:
df_list = df_train.values.tolist()
df_array = np.array(df_list)
df_array

df_list2 = df_mul.values.tolist()
df_array2 = np.array(df_list2)

In [4]:
y_train = np.array(df2['label'])

In [5]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [31]:
model = Sequential()
model.add(Embedding(9294, 50))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

In [32]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [33]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(df_array2, y_train, epochs=15, callbacks=[es, mc], batch_size=5, validation_split=0.2)

Epoch 1/15
196/196 [==============================] - 2s 6ms/step - loss: 0.6923 - acc: 0.5456 - val_loss: 0.7852 - val_acc: 0.0000e+00

Epoch 00001: val_acc improved from -inf to 0.00000, saving model to best_model.h5
Epoch 2/15
196/196 [==============================] - 1s 4ms/step - loss: 0.6458 - acc: 0.6258 - val_loss: 1.3121 - val_acc: 0.0490

Epoch 00002: val_acc improved from 0.00000 to 0.04898, saving model to best_model.h5
Epoch 3/15
196/196 [==============================] - 1s 4ms/step - loss: 0.5728 - acc: 0.7161 - val_loss: 0.9865 - val_acc: 0.2939

Epoch 00003: val_acc improved from 0.04898 to 0.29388, saving model to best_model.h5
Epoch 4/15
196/196 [==============================] - 1s 4ms/step - loss: 0.5646 - acc: 0.7322 - val_loss: 0.8496 - val_acc: 0.4898

Epoch 00004: val_acc improved from 0.29388 to 0.48980, saving model to best_model.h5
Epoch 5/15
196/196 [==============================] - 1s 4ms/step - loss: 0.5172 - acc: 0.7576 - val_loss: 0.8592 - val_acc: 0.

In [34]:
model.save('SNU_LSTM_Model.h5')

In [25]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 200)         1858800   
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               168448    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 2,027,377
Trainable params: 2,027,377
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 100)         929400    
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 971,705
Trainable params: 971,705
Non-trainable params: 0
_________________________________________________________________


In [1]:
import pandas as pd
from konlpy.tag import Okt
import numpy as np
import boto3
import sys
from gensim.models import FastText
import os, csv, math, codecs
from tqdm import tqdm
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Embedding, Dense, LSTM, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


MAX_NB_WORDS = 100000

# =========================================================================
# Vectorizer의 argument인 tokenizer에 KoNLPy의 pos 함수로 대체.
class MyTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger

    def __call__(self, sent):
        pos = self.tagger.pos(sent)
        clean_words = []  # 정제된 단어 리스트
        for word in pos:
            # word[1]은 품사를 의미하며, 여기서는 조사, 문장기호, 접두사, Foreign('\n'을 빼주기 위함)인 것은 제외시킴.
            if word[1] not in ['Josa', 'Punctuation', 'Suffix', 'Foreign']:
                if len(word[0]) >= 2:  # 한 글자인 단어들도 의미가 없는 경우가 많으므로 일단 제외.
                    #if word[0] not in ['있다', '했다', '한다', '없다', '된다']:
                    clean_words.append(word[0])
        return clean_words

# =========================================================================

embeddings_index = {}
f = codecs.open('/home/ubuntu/FastText/wiki.ko.vec', encoding='utf-8')

for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('found %s word vectors' % len(embeddings_index))

# ==============================================================
if sys.version_info[0] < 3:
    from io import StringIO # Python 2.x
else:
    from io import StringIO # Python 3.x

# get your credentials from environment variables
aws_id = ''
aws_secret = ''

client = boto3.client('s3', aws_access_key_id=aws_id,
        aws_secret_access_key=aws_secret)

bucket_name = 'snucsv'

object_key = 'SNU_Data_1200.csv'
csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
body = csv_obj['Body']
csv_string = body.read().decode('cp949')

# load data
train_df = pd.read_csv(StringIO(csv_string))
label_names = ["label"]
y_train = train_df[label_names].values

# test_df = pd.read_csv('D:\pythonProject\FakeNewsFiltering\SNU_Validation_200.csv', encoding='CP949')
# test_df = test_df.fillna('_NA_')

# print(train_df)
train_df['doc_len'] = train_df['document'].apply(lambda words: len(words.split(" ")))
max_seq_len = np.round(train_df['doc_len'].mean() + train_df['doc_len'].std()).astype(int)
# ==================================================================================
from sklearn.utils import shuffle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

my_Tokenizer = MyTokenizer(Okt())

raw_docs_train = train_df['document'].tolist()
# raw_docs_test = test_df['document'].tolist()
# print(raw_docs_test)
num_classes = len(label_names)

processed_docs_train = []
# processed_docs_test = []

for doc in tqdm(raw_docs_train):
    tokens = my_Tokenizer(doc)
    processed_docs_train.append(tokens)

# for doc in tqdm(raw_docs_test):
#     tokens = my_Tokenizer(doc)
#     processed_docs_test.append(tokens)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
# tokenizer.fit_on_texts(processed_docs_train + processed_docs_test)
tokenizer.fit_on_texts(processed_docs_train)
# tokenizer.fit_on_texts(processed_docs_train+proccessed_docs_test)
word_seq_train = tokenizer.texts_to_sequences(processed_docs_train)
# word_seq_test = tokenizer.texts_to_sequences(processed_docs_test)
word_index = tokenizer.word_index
print("dictionary size : ", len(word_index))

word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
# word_seq_test = sequence.pad_sequences(word_seq_test, maxlen=max_seq_len)
# ====================================================================================
# training params
batch_size = 256
num_epochs = 40

# model parameters
num_filters = 64
embed_dim = 300
weight_decay = 1e-4

# embedding matrix

words_not_found = []
nb_words = min(MAX_NB_WORDS, len(word_index)+1)
embedding_matrix = np.zeros((nb_words, embed_dim))

# print(word_index)
for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
        # cc.ko.300.vec에서 찾지 못한 단어들의 리스트.
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
print("sample words not found: ", np.random.choice(words_not_found, 10))



/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
879131it [01:24, 10411.96it/s]


found 879130 word vectors


100%|██████████| 1224/1224 [00:07<00:00, 157.57it/s]


dictionary size :  3999
number of null word embeddings: 354
sample words not found:  ['6조' '82~88년' '2020년' '1억' '2017년' '1천' '미프진' '10년' '6.13' '360']


In [3]:
# ========================================================================================
from keras.layers import BatchNormalization
import tensorflow as tf

model = tf.keras.Sequential()

model.add(Embedding(nb_words, embed_dim, input_length=max_seq_len, weights=[embedding_matrix], trainable=False))

model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(1, activation='sigmoid'))
model.summary()
# ================================================================================
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

es_callback = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(word_seq_train_shuffle, y_train_shuffle, batch_size=128,
          epochs=num_epochs, validation_split=0.3, callbacks=[es_callback], shuffle=False)

# predictions = model.predict_classes(word_seq_test)

Extension horovod.torch has not been built: /home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-37m-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.
[2021-08-10 10:12:13.802 ip-172-31-2-138:1564 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-08-10 10:12:14.178 ip-172-31-2-138:1564 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 300)           1200000   
_________________________________________________________________
dropout (Dropout)            (None, 10, 300)           0         
___________

In [2]:
word_seq_all_shuffle = train_df.sample(frac=1).reset_index(drop=True)
word_seq_train_shuffle = word_seq_all_shuffle['document'].tolist()
y_train_shuffle = word_seq_all_shuffle['label'].values

processed_docs_train_shuffle = []

for doc in tqdm(word_seq_train_shuffle):
    tokens = my_Tokenizer(doc)
    processed_docs_train_shuffle.append(tokens)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
tokenizer.fit_on_texts(processed_docs_train_shuffle)
word_seq_train_shuffle = tokenizer.texts_to_sequences(processed_docs_train_shuffle)
word_seq_train_shuffle = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)

100%|██████████| 1224/1224 [00:02<00:00, 502.09it/s]


In [9]:
# word_seq_train_tuple = tuple(word_seq_train)

In [3]:
# from sklearn.model_selection import KFold
# from sklearn.model_selection import cross_val_score

# seed = 7
# kfold = KFold(n_splits=3, shuffle=True, random_state=seed)
# result = cross_val_score(model, word_seq_train_shuffle, y_train_shuffle, scoring='accuracy', cv=kfold)

# print('교차 검증별 정확도:', np.round(result,4))
# print('평균 검증 정확도:', np.round(np.mean(result), 4))

In [4]:
model.save('SNU_LSTM_Model_shuffle.h5')

In [9]:
response = ['현재 AI기술은 불완전하기 때문에 가짜뉴스 완전히 걸러낼 수 없다']
tokens_response = []

for i in response:
    tokens = my_Tokenizer(response[0])
    tokens_response.append(tokens)
    print(tokens_response)

word_seq_response = tokenizer.texts_to_sequences(tokens_response)
word_seq_response = sequence.pad_sequences(word_seq_response, maxlen=max_seq_len)
score = model.predict(word_seq_response)
score

[['현재', 'AI', '기술', '불완전하기', '때문', '가짜', '뉴스', '완전히', '걸러', '없다']]


array([[0.5054651]], dtype=float32)

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)